In [2]:
import numpy as np
import pydub
from scipy.io import wavfile
from IPython.display import Audio
#most likely need to !pip install pydub
from pydub import AudioSegment

In [11]:
#audio segments
dot = AudioSegment.from_wav('dot.wav')
dash = AudioSegment.from_wav('dash.wav')
space = AudioSegment.from_wav('silent_half-second.wav')
silent = AudioSegment.from_wav('silent_quarter-second.wav')

#list for conversion
morse_dict = { 'A':'.-', 'B':'-...',
               'C':'-.-.', 'D':'-..', 'E':'.',
               'F':'..-.', 'G':'--.', 'H':'....',
               'I':'..', 'J':'.---', 'K':'-.-',
               'L':'.-..', 'M':'--', 'N':'-.',
               'O':'---', 'P':'.--.', 'Q':'--.-',
               'R':'.-.', 'S':'...', 'T':'-',
               'U':'..-', 'V':'...-', 'W':'.--',
               'X':'-..-', 'Y':'-.--', 'Z':'--..'}

#conversion function
def eng2morse(english):
    morse_text = ''
    morse_sound = AudioSegment.empty() 
    i = 0
    
    #loops through word
    #adds space after each letter, 2 spaces for words
    while(i<len(english)):
        if(english[i] != " "):
            morse_text += morse_dict[english[i]] + ' '
        else:
            morse_text += ' '
        i+=1

    j=0
    #loops through morse, adds segments to empty audio file for each dot or dash
    # 0.25 second of silence between dots and dashes
    # 0.75 seconds between letters
    #these are true to morse units of measurement
    while(j<len(morse_text)):
        if(morse_text[j] == "."):
            morse_sound += dot + silent
            #print('dot')
        elif(morse_text[j] == "-"):
            morse_sound += dash + silent
            #print('dash')
        else:
            if(morse_text[j-1] == " "):  #if last letter was a space, meaning 2 spaces in a row, then must add more time between words
                morse_sound += space + space + silent
                #print('word space')
            elif(j == len(morse_text)-1):  #last letter, no need for extra silence after
                break
            else:
                morse_sound += silent
                #print('letter space')
        j+=1
    
    #exporting full audio signal
    morse_sound.export("sound.wav", format="wav")
    fs, data = wavfile.read('sound.wav')
    return morse_text, fs, data

    #main for running functions
def mainTranslate():
    english = input("Word to be translated(from edge detection system): ")
    output, output_fs, output_data= eng2morse(english.upper())
    print(output)
    wavfile.write('output.wav', output_fs, output_data)
    
mainTranslate()
Audio('output.wav')

Word to be translated(from edge detection system): help me
.... . .-.. .--.  -- . 


In [13]:
##Did not have time yet/don't think were going to use a Spectrogram, but if we do this is where I would begin to visualize it

import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import read, write
from numpy.fft import fft, ifft
FRAME_SIZE = 1024
ZP_FACTOR = 2
FFT_SIZE = FRAME_SIZE * ZP_FACTOR
PAD = FFT_SIZE - FRAME_SIZE
################## YOUR CODE HERE ######################
def ece420ProcessFrame(frame):
    curFft = np.zeros(FFT_SIZE)
    w_result = np.zeros(FFT_SIZE)
    for i in range(FRAME_SIZE): # windowing by convolution 
        win = 0.54 - 0.46 * np.cos(2*np.pi*i/(FRAME_SIZE-1))
        w_result[i] = win*frame[i] #window
        pad = np.pad(w_result, (0,PAD), 'constant') #zero padding 
        square = np.square(np.abs(fft(w_result, FFT_SIZE))) # FFT and magnitude squared of fft
        curFft = np.log10(square) #Log()
    return curFft[:FRAME_SIZE]
################# GIVEN CODE BELOW #####################
Fs, data = read('output.wav')
numFrames = int(len(data) / FRAME_SIZE)
bmp = np.zeros((numFrames, FRAME_SIZE))
for i in range(numFrames):
    frame = data[i * FRAME_SIZE : (i + 1) * FRAME_SIZE]
    curFft = ece420ProcessFrame(frame)
    bmp[i, :] = curFft
bmp_scaled = bmp/np.max(bmp) #scaling the result 
plt.figure()
plt.pcolormesh(bmp_scaled.T, vmin=0, vmax = 1)
plt.axis('tight')
plt.show()

from IPython.display import Audio
Audio(data,rate=Fs)

ValueError: setting an array element with a sequence.